# Экзамен по SQL

In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("test_tbankd.db")
cursor = conn.cursor()

In [5]:
customers_df = pd.DataFrame({"cust_id": ["C1", "C2", "C3"],
                             "cust_name": ["N1", "N2", "N3"],
                             "city": ["Москва", "Санкт-Петербург", "Казань"],
                             "singup_date": ["2023-01-05",
                                             "2023-01-12",
                                             "2023-01-20"]})

In [50]:
orders_df = pd.DataFrame({"order_id": ["O1", "O2", "O3", "O4", "O5", "O6", "07"],
                          "cust_id": ["C1", "C2", "C1", "C3", "C2", "C3", "C1"],
                          "restaurant": ['ПиццаНЯМ', 'СушиДом', 'БургерСпа', 'Лапша-Бар', 'СушиДом', 'Лапша-Бар', 'ЛОСЬ'],
                          "order_date": ["2023-02-01", "2023-02-03",
                                         "2023-02-10", "2023-02-12", "2023-02-12", "2023-02-12", '2024-02-12'],
                          "total": [1200.00, 1800.50, 750.00, 900.00, 1950.00, 100.00, 600.00],
                          "status": ["completed", "cancelled", "completed", 'completed', 'completed', 'completed', 'cancelled']})

In [8]:
feedback_df = pd.DataFrame({"feedback_id": ["f1", "f2", "f3"],
                          "order_id": ["01", "03", "04"],
                          "rating": [5, 4, 5],
                          "feedback_date": ["2023-02-01", "2023-02-11", "2023-02-12"]})

In [9]:
feedback_df.head()

,feedback_id,order_id,rating,feedback_date
0,f1,01,5,2023-02-01
1,f2,03,4,2023-02-11
2,f3,04,5,2023-02-12


In [10]:
feedback_df.to_sql("feedback", conn, index=False)

3

In [53]:
orders_df.to_sql("orders", conn, index=False)

7

In [12]:
customers_df.to_sql("customers", conn, index=False)

3

In [55]:
text = """
    SELECT COUNT(order_id)
    FROM orders
    WHERE status == 'completed';
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0
0,5


In [52]:
# cursor.execute("DROP TABLE orders;")
# conn.commit()

In [54]:
"""
Напишите запрос, который выведет город с наибольшей суммарной 
выручкой по завершённым заказам. 
Если таких городов несколько, выберите тот, 
чьё название идёт первым по алфавиту.
"""

text = """
    WITH cities_total AS
    (SELECT c.city AS city, SUM(o.total) AS total
    FROM customers c INNER JOIN orders o ON c.cust_id = o.cust_id
    WHERE o.status == 'completed'
    GROUP BY c.city)
    SELECT city
    FROM cities_total
    WHERE total = (SELECT MAX(total)
                   FROM cities_total)
    ORDER BY city
    LIMIT 1;
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0
0,Москва


In [43]:
"""
Напишите запрос, который выведет среднюю оценку 
по всем отзывам (округлите до 2 знаков)

"""

text = """
    SELECT ROUND(AVG(rating), 2) as avg_rating
    FROM feedback;
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0
0,4.67


In [81]:
"""
Напишите запрос, который выведет ресторан 
с наибольшим числом повторных клиентов.
Повторный клиент — это клиент, у которого в данном ресторане 
как минимум 2 завершённых заказа (status = 'completed').
При равенстве числа повторных клиентов:
 выбирайте ресторан с большим количеством завершённых заказов;
при новом равенстве — ресторан с названием, 
идущим раньше по алфавиту.

"""

text = """
    WITH clients_stats AS (
    SELECT restaurant, cust_id, COUNT(*) AS repeated_orders_cnt
    FROM orders
    WHERE status = 'completed'
    GROUP BY restaurant, cust_id
    HAVING repeated_orders_cnt >= 2),

    repeated_clients_stats AS (
    SELECT restaurant, COUNT(cust_id) AS repeated_clients_cnt
    FROM clients_stats
    GROUP BY restaurant),

    rest_stats AS (
    SELECT restaurant, COUNT(order_id) AS comp_orders_cnt
    FROM orders
    WHERE status = 'completed'
    GROUP BY restaurant),

    total_stats AS (
    SELECT DISTINCT o.restaurant as restaurant, COALESCE(cs.repeated_clients_cnt, 0) as repeated_clients_cnt,
        COALESCE(rs.comp_orders_cnt, 0) as comp_orders_cnt
    FROM orders o LEFT JOIN repeated_clients_stats cs ON o.restaurant = cs.restaurant
        LEFT JOIN rest_stats rs ON o.restaurant = rs.restaurant)
    SELECT restaurant
    FROM total_stats
    WHERE repeated_clients_cnt = (SELECT MAX(repeated_clients_cnt)
                                 FROM total_stats)
    ORDER BY comp_orders_cnt DESC, restaurant
    LIMIT 1;
"""

cursor.execute(text)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0
0,Лапша-Бар
